In [151]:
from confluent_kafka import Consumer, KafkaException
from config import config
import pandas as pd
import json
from pymongo import MongoClient

In [152]:
topic = 'ds23m003_DSI_Abschlussprojekt'

In [153]:
def process_data(data,collection):
    global df
    data_dict = json.loads(data)


    Dis_code = data_dict["DISTRICT_CODE"]
    ref_year = data_dict["REF_YEAR"]

    total_einkommen = data_dict["INC_TOT_VALUE"]
    Male_einkommen = data_dict["INC_MAL_VALUE"]
    Female_einkommen= data_dict["INC_FEM_VALUE"]
    Geburten = data_dict["BIR"]

    data_to_insert = {
        "District_code": Dis_code,
        "Year": ref_year,
        "Total_Salary": total_einkommen,
        "Male_Salary": Male_einkommen,
        "Female_Salary": Female_einkommen,
        "Births": Geburten
    }

    collection.insert_one(data_to_insert)

In [154]:
def set_consumer_configs():
    config['group.id'] = 'hello_group'
    config['auto.offset.reset'] = 'earliest'
    config['enable.auto.commit'] = False

In [155]:
def set_consumer_configs():
    config['group.id'] = 'hello_group'
    config['auto.offset.reset'] = 'earliest'
    config['enable.auto.commit'] = False

In [156]:
def assignment_callback(consumer, partitions):
    for p in partitions:
        print(f'Assigned to {p.topic}, partition {p.partition}')

In [157]:
def consume_data(consumer,collection):
    consumer.subscribe([topic])

    try:
        while True:
            event = consumer.poll(1.0)

            if event is None:
                continue

            if event.error():
                raise KafkaException(event.error())
            else:
                val = event.value().decode('utf8')
                partition = event.partition()
                process_data(val,collection)
                print(f'Received: {val} from partition {partition}')
            consumer.commit()

    except KeyboardInterrupt:
        print('Canceled by user.')
    except Exception as e:
        print(f'Error in consume_data: {e}')
    finally:
        consumer.close()

In [158]:
try:
    client = MongoClient("mongodb://localhost:27017/")
    print("connected successfully")
except:
    print ("could not connect to mongoDB")

db = client["DSI_Abschlussprojekt"]

collection=db.create_collection('Geburten_nach_Einkommen_Wien')


connected successfully


In [159]:
set_consumer_configs()
consumer = Consumer(config)
consumer.subscribe([topic], on_assign=assignment_callback)
consume_data(consumer,collection)

Received: {"Unnamed: 0": 0, "DISTRICT_CODE": 90100, "REF_YEAR": 2002, "INC_TOT_VALUE": 25463, "INC_MAL_VALUE": 31961, "INC_FEM_VALUE": 18536, "BIR": 130} from partition 2
Received: {"Unnamed: 0": 1, "DISTRICT_CODE": 90200, "REF_YEAR": 2002, "INC_TOT_VALUE": 16439, "INC_MAL_VALUE": 18301, "INC_FEM_VALUE": 14282, "BIR": 981} from partition 3
Received: {"Unnamed: 0": 2, "DISTRICT_CODE": 90300, "REF_YEAR": 2002, "INC_TOT_VALUE": 18701, "INC_MAL_VALUE": 21444, "INC_FEM_VALUE": 15804, "BIR": 761} from partition 4
Received: {"Unnamed: 0": 3, "DISTRICT_CODE": 90400, "REF_YEAR": 2002, "INC_TOT_VALUE": 20325, "INC_MAL_VALUE": 23641, "INC_FEM_VALUE": 16876, "BIR": 259} from partition 1
Received: {"Unnamed: 0": 4, "DISTRICT_CODE": 90500, "REF_YEAR": 2002, "INC_TOT_VALUE": 16258, "INC_MAL_VALUE": 17937, "INC_FEM_VALUE": 14395, "BIR": 557} from partition 1
Received: {"Unnamed: 0": 5, "DISTRICT_CODE": 90600, "REF_YEAR": 2002, "INC_TOT_VALUE": 18724, "INC_MAL_VALUE": 21260, "INC_FEM_VALUE": 16068, "BI

[{'_id': ObjectId('65a4227b652a4ea3ea112c0b'),
  'District_code': 90100,
  'Year': 2002,
  'Total_Salary': 25463,
  'Male_Salary': 31961,
  'Female_Salary': 18536,
  'Births': 130},
 {'_id': ObjectId('65a4227b652a4ea3ea112c0c'),
  'District_code': 90200,
  'Year': 2002,
  'Total_Salary': 16439,
  'Male_Salary': 18301,
  'Female_Salary': 14282,
  'Births': 981},
 {'_id': ObjectId('65a4227b652a4ea3ea112c0d'),
  'District_code': 90300,
  'Year': 2002,
  'Total_Salary': 18701,
  'Male_Salary': 21444,
  'Female_Salary': 15804,
  'Births': 761},
 {'_id': ObjectId('65a4227b652a4ea3ea112c0e'),
  'District_code': 90400,
  'Year': 2002,
  'Total_Salary': 20325,
  'Male_Salary': 23641,
  'Female_Salary': 16876,
  'Births': 259},
 {'_id': ObjectId('65a4227b652a4ea3ea112c0f'),
  'District_code': 90500,
  'Year': 2002,
  'Total_Salary': 16258,
  'Male_Salary': 17937,
  'Female_Salary': 14395,
  'Births': 557},
 {'_id': ObjectId('65a4227b652a4ea3ea112c10'),
  'District_code': 90600,
  'Year': 2002,
 